In [ ]:
# !pip install SMOTE

In [ ]:
# !pip install imbalanced-learn

In [ ]:
# !pip install -U scikit-learn imbalanced-learn

In [ ]:
# pip install tensorflow

In [ ]:
# pip install xgboost

In [ ]:
# pip install gensim

In [ ]:
# pip install numpy

In [ ]:
# pip install pandas

In [ ]:
# pip install matplotlib

In [ ]:
# pip install transformers

In [ ]:
# pip install nltk

In [ ]:
# import nltk

In [ ]:
# nltk.download('stopwords')

In [ ]:
# nltk.download('punkt')

In [ ]:
# nltk.download('wordnet')

In [ ]:
# pip install gensim

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
# import keras.preprocessing.text Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense
from keras.models import Sequential
from imblearn.under_sampling import TomekLinks
from imblearn.over_sampling import RandomOverSampler
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif
import xgboost as xgb
from gensim.models import KeyedVectors
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MaxAbsScaler
from sklearn.feature_extraction.text import HashingVectorizer
# import spacy
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from sklearn.preprocessing import Normalizer
from gensim.models import KeyedVectors
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import re
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from gensim.models import KeyedVectors
import random
import tensorflow as tf
import os
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
from sklearn.model_selection import cross_val_score

In [ ]:
train = pd.read_csv("./train.csv")
test = pd.read_csv("./test.csv")

In [ ]:
train.describe()

In [ ]:
def preprocess(text):
    pattern = r"[^\w\s]"
    text = [''.join([char if char.isalpha() else ' ' for char in word]) for word in text.split()]
    text = ' '.join(text)
    text =  re.sub(pattern," ",text)
     # used word_tokenize function to tokenize the text, gives list

    tokenized_text = word_tokenize(text.lower())
    # get the stop words
    stop_words = set(stopwords.words('english'))
    # removed stop words
    tokenized_text = [word for word in tokenized_text if word not in stop_words]
    # applying stemming
    # stemmer = PorterStemmer()
    # tokenized_text = [stemmer.stem(word) for word in tokenized_text]
    # applying lemmatization
    lemmatizer = WordNetLemmatizer()
    tokenized_text = [lemmatizer.lemmatize(word) for word in tokenized_text]

    preprocessed_text = ' '.join(tokenized_text)

    return preprocessed_text

In [ ]:
def reset_random_seeds(SEED=42):
   os.environ['PYTHONHASHSEED']=str(SEED)
   tf.random.set_seed(SEED)
   np.random.seed(SEED)
   random.seed(SEED)
reset_random_seeds(SEED=42)

In [ ]:
train.head(20)

In [ ]:
train.isnull().sum()

In [ ]:
train.isnull().sum()

In [ ]:
train.duplicated().sum()

In [ ]:
train = train.drop_duplicates()

In [ ]:
train.shape

In [ ]:
class_counts = train['overall'].value_counts()

plt.bar(class_counts.index, class_counts.values)
plt.xlabel('Class')
plt.ylabel('Number of Samples')
plt.title('Histogram of Sample Counts per Class')
plt.show()

In [ ]:
test.describe()

In [ ]:
test.head(20)

In [ ]:
test.isnull().sum()

In [ ]:
test.duplicated().sum()

In [ ]:
test.shape

In [ ]:
train["Review"]= train["Review"].astype(str)


In [ ]:
train["Review"] = train["Review"].apply(preprocess)

In [ ]:
train

In [ ]:
y = train["overall"]
X_train, X_test, y_train, y_test = train_test_split(train["Review"], y, stratify=y,test_size=0.3, random_state=42)
print('Original class distribution:', Counter(y_train))

In [ ]:
X_train.shape

In [ ]:
def create_pipeline(classifiers,vectorizer, normalizer=None, reduction= False, feature_selection=True,k=1000,balancing=False ):
     # start building the pipeline, first step -> vectorization
    step =[("vectorizer",vectorizer)]
     # if dimensionality reduction is requested add a TruncatedSVD step to the pipeline
    if normalizer:
        step.append(("normalizer",normalizer))
    # if feature selection is requested add a SelectKBest step to the pipeline
    if feature_selection:
        step.append(('feature_selection', SelectKBest(chi2, k=k)))
    # add the classifier to the pipeline
    step.append(("classifier",classifiers))

        # pipeline object with the steps prepared above and assign it to the corresponding classifier name in the models dict
    models = Pipeline(step)
    return models

## Logistic Regression

### Comparing Ngram Features

#### Unigram

In [ ]:
def logistic_hyperparameter(ngram):
    space = {
        'C': hp.loguniform('C', np.log(0.01), np.log(10)),
        'penalty': hp.choice('penalty', ['l2']),
        'k':hp.choice('k', np.arange(8000, 35000, 1000, dtype=int)),
        'class_weight': hp.choice('class_weight', ['balanced', {1: 7, 2: 12, 3: 9, 4: 8, 5: 7}]),
        'solver': hp.choice('solver',['newton-cg', 'lbfgs', 'liblinear','saga']),
        'max_iter': hp.choice('max_iter',np.arange(1000, 8000, 500, dtype=int))
    }

    def objective(params):
        k = params.pop('k')
        classifier = LogisticRegression(**params)
        vectorizer = TfidfVectorizer(ngram_range=ngram)
        
        model = create_pipeline(classifier, vectorizer, feature_selection=True, k=k)
        score = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=5).mean()
        print(f"\nTrial completed:")
        print(f"Params: {params} features:{k}")
        print(f"Accuracy: {score}, Loss: {-score}")
        return {'loss': -score, 'status': STATUS_OK}

    trials = Trials()
    best = fmin(fn=objective,
                space=space,
                algo=tpe.suggest,
                max_evals=50,
                trials=trials)
    logistic = best
    print("Best hyperparameters:", best)
    return logistic

In [ ]:
best_uni = logistic_hyperparameter((1,1))

In [ ]:
# class_weights = {1: 7, 2: 12, 3: 9, 4: 8, 5: 7}
# classifier = LogisticRegression(max_iter=1000,penalty="l2",C=0.13,solver= "liblinear" ,class_weight=class_weights)
# model = create_pipeline(classifier, TfidfVectorizer(ngram_range=(1,1)),k=15000)
# model.fit(X_train,y_train)
# y_pred = model.predict(X_test)
# print(classification_report(y_pred,y_test))

In [ ]:
best_bigram = logistic_hyperparameter((2,2))

In [ ]:
# class_weights = {1: 7, 2: 12, 3: 9, 4: 8, 5: 7}
# classifier = LogisticRegression(max_iter=1000,penalty="l2",C=0.13,solver= "liblinear" ,class_weight=class_weights)
# model = create_pipeline(classifier, TfidfVectorizer(ngram_range=(2,2)),k=20000)
# model.fit(X_train,y_train)
# y_pred = model.predict(X_test)
# print(classification_report(y_pred,y_test))

In [ ]:
best_trigram = logistic_hyperparameter((3,3))

In [ ]:
# class_weights = {1: 7, 2: 12, 3: 9, 4: 8, 5: 7}
# classifier = LogisticRegression(max_iter=1000,penalty="l2",C=0.13,solver= "liblinear" ,class_weight=class_weights)
# model = create_pipeline(classifier, TfidfVectorizer(ngram_range=(3,3)))
# model.fit(X_train,y_train)
# y_pred = model.predict(X_test)
# print(classification_report(y_pred,y_test))

In [ ]:
best_n_gram = logistic_hyperparameter((1,3))

In [ ]:
# class_weights = {1: 7, 2: 12, 3: 9, 4: 8, 5: 7}
# classifier = LogisticRegression(max_iter=1000,penalty="l2",C=0.13,solver= "liblinear" ,class_weight=class_weights)
# model = create_pipeline(classifier, TfidfVectorizer(ngram_range=(1,3)),k=24500)
# model.fit(X_train,y_train)
# y_pred = model.predict(X_test)
# print(classification_report(y_pred,y_test))

## XGBoost

In [ ]:
# sampling_strategy = {i: 23000 for i in range(1, 4)}
# undersampler = RandomOverSampler(sampling_strategy=sampling_strategy, random_state=42)
# X_train, y_train= undersampler.fit_resample(X_train, y_train)

# y_train_adjusted = y_train-1
# y_test = y_test-1

### NGram

In [ ]:
# space = {
#     'max_depth': hp.choice('max_depth', np.arange(3, 11, dtype=int)),
#     'n_estimators': hp.choice('n_estimators', np.arange(100, 1001, 100, dtype=int)),
#     'learning_rate': hp.uniform('learning_rate', 0.01, 0.3),
#     'subsample': hp.uniform('subsample', 0.5, 1.0),
#     'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0),
#     'min_child_weight': hp.choice('min_child_weight', np.arange(1, 11, dtype=int)),
#     'reg_lambda': hp.uniform('reg_lambda', 1, 100),
#     'k': hp.choice('k', np.arange(8000, 17000, 500, dtype=int))
# }

# def objective(params):
#     k = params.pop('k')
#     feature_selector = SelectKBest(chi2, k=k)
#     X_train_sel = feature_selector.fit_transform(X_train, y_train)
#     X_test_sel = feature_selector.transform(X_test)
#     clf = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', **params)
#     score = cross_val_score(clf, X_train_sel, y_train_adjusted, scoring='accuracy', cv=5).mean()
#     print(f"\nTrial completed:")
#     print(f"Params: {params} features:{k}")
#     print(f"Accuracy: {score}, Loss: {-score}")
#     return {'loss': -score, 'status': STATUS_OK}
# # Run the optimization
# trials = Trials()
# best = fmin(fn=objective,
#             space=space,
#             algo=tpe.suggest,
#             max_evals=50,
#             trials=trials)

# print("Best hyperparameters:", best)

In [ ]:
# params = {'colsample_bytree': 0.8230008586447808,
#  'learning_rate': 0.1909617073607323,
#  'max_depth': 10,
#  'min_child_weight': 4,
#  'n_estimators': 1000,
#  'reg_lambda': 4.163618957089685,
#  'subsample': 0.59102293915828 }

# classifier =  xgb.XGBClassifier(eval_metric='mlogloss', **params)

# model = create_pipeline(classifier, TfidfVectorizer(ngram_range=(1,3)),k=10500)
# model.fit(X_train,y_train_adjusted)
# y_pred = model.predict(X_test)
# print(classification_report(y_pred,y_test))
# y_test = y_test+1

### Googles Word2Vec

In [ ]:
# google_word2vec = KeyedVectors.load_word2vec_format("../Amazon-Reviews/GoogleNews-vectors-negative300.bin", binary=True)

In [ ]:
# def document_vector(word2vec_model, doc_tokens):
#     tokens = [token for token in doc_tokens if token in word2vec_model.key_to_index]
#     if not tokens:
#         return np.zeros(word2vec_model.vector_size)
#     doc_vector = np.mean(word2vec_model[tokens], axis=0)
#     return doc_vector

In [ ]:
# X_train = np.array([document_vector(google_word2vec,doc) for doc in X_train])
# X_test = np.array([document_vector(google_word2vec,doc) for doc in X_test])
# sampling_strategy = {i: 23000 for i in range(1, 4)}
# undersampler = RandomOverSampler(sampling_strategy=sampling_strategy, random_state=42)
# X_train_under, y_train_under = undersampler.fit_resample(X_train, y_train)

In [ ]:
# y_train_under = y_train_under-1

In [ ]:
# space = {
#     'max_depth': hp.choice('max_depth', np.arange(3, 11, dtype=int)),
#     'n_estimators': hp.choice('n_estimators', np.arange(100, 1001, 100, dtype=int)),
#     'learning_rate': hp.uniform('learning_rate', 0.01, 0.3),
#     'subsample': hp.uniform('subsample', 0.5, 1.0),
#     'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0),
#     'min_child_weight': hp.choice('min_child_weight', np.arange(1, 11, dtype=int)),
#     'reg_lambda': hp.uniform('reg_lambda', 1, 100),
# }

# def objective(params):
#     clf = xgb.XGBClassifier(eval_metric='mlogloss', **params)
#     score = cross_val_score(clf, X_train_under, y_train_under, scoring='accuracy', cv=5).mean()
#     print(f"\nTrial completed:")
#     print(f"Params: {params} features:{k}")
#     print(f"Accuracy: {score}, Loss: {-score}")
#     return {'loss': -score, 'status': STATUS_OK}
# # Run the optimization
# trials = Trials()
# best = fmin(fn=objective,
#             space=space,
#             algo=tpe.suggest,
#             max_evals=50,
#             trials=trials)

# print("Best hyperparameters:", best)

## Random Forest Classifier

In [ ]:
train = pd.read_csv("./train.csv")
test = pd.read_csv("./test.csv")
train = train.drop_duplicates()
train["Review"]= train["Review"].astype(str)
train["Review"] = train["Review"].apply(preprocess)
y = train["overall"]
X_train, X_test, y_train, y_test = train_test_split(train["Review"], y, stratify=y,test_size=0.3, random_state=42)
print('Original class distribution:', Counter(y_train))

In [ ]:
space = {
    'n_estimators': hp.choice('n_estimators', np.arange(100, 1501, 100, dtype=int)),
    'max_depth': hp.quniform('max_depth', 5, 100, 5),  
    'min_samples_split': hp.quniform('min_samples_split', 2, 20, 1), 
    'min_samples_leaf': hp.quniform('min_samples_leaf', 1, 10, 1), 
    'max_features': hp.choice('max_features', [ 'sqrt', 'log2', None]),
    'k': hp.choice('k', np.arange(8000, 20000, 1000, dtype=int)),
}

def objective(params):
    params['max_depth'] = int(params['max_depth'])
    params['min_samples_split'] = int(params['min_samples_split'])
    params['min_samples_leaf'] = int(params['min_samples_leaf'])

    k = params.pop('k')
    classifier = RandomForestClassifier(**params)
    vectorizer = CountVectorizer()

    model = create_pipeline(classifier, vectorizer,Normalizer(norm='l2') ,feature_selection=True, k=k)
    score = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=5).mean()
    print(f"\nTrial completed:")
    print(f"Params: {params} features:{k}")
    print(f"Accuracy: {score}, Loss: {-score}")
    return {'loss': -score, 'status': STATUS_OK}

trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=50,
            trials=trials)
random_forest_best = best
print(random_forest_best)
print("Best hyperparameters:", best)

In [ ]:
# classifier =  RandomForestClassifier()
# model = create_pipeline(classifier,CountVectorizer(),Normalizer(norm='l2'))
# model.fit(X_train,y_train)
# y_pred = model.predict(X_test)
# print(classification_report(y_pred,y_test))
# y_pred = model.predict(X_train)
# print(classification_report(y_pred,y_train))

## SGD

In [ ]:
space = {
    'alpha': hp.loguniform('alpha', np.log(0.0001), np.log(0.1)),
    'penalty': hp.choice('penalty', ['l2', 'l1', 'elasticnet']),
    'loss': hp.choice('loss', ['hinge', 'log_loss', 'modified_huber']),
    'max_iter': hp.choice('max_iter', np.arange(1000, 12000, 1000, dtype=int)),
    'learning_rate': hp.choice('learning_rate', ['constant', 'optimal', 'invscaling', 'adaptive']),
    'eta0': hp.loguniform('eta0', np.log(0.01), np.log(1)),
    'k': hp.choice('k', np.arange(8000, 20000, 1000, dtype=int)),
}

def objective(params):
    k = params.pop('k')
    classifier = SGDClassifier(**params)
    vectorizer = CountVectorizer(binary = True)

    model = create_pipeline(classifier, vectorizer,Normalizer(norm='l2') ,feature_selection=True, k=k)
    score = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=5).mean()
    print(f"\nTrial completed:")
    print(f"Params: {params} features:{k}")
    print(f"Accuracy: {score}, Loss: {-score}")
    return {'loss': -score, 'status': STATUS_OK}

trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=50,
            trials=trials)

print("Best hyperparameters:", best)

# Sequence Model

## Bi-Directional LSTM

## LSTM

## Transformers